In [ ]:
# Import functions 
import sys
sys.path.insert(0, '..')
from utils import *  

In [ ]:
# Starter code class that handles the fancy stuff. No need to modify this! 
class Racecar:
    SCAN_TOPIC = "/scan"
    IMAGE_TOPIC = "/camera"
    DRIVE_TOPIC = "/drive"
    
    def __init__(self):
        self.sub_scan = rospy.Subscriber(self.SCAN_TOPIC, LaserScan, callback=self.scan_callback)
        self.sub_image = rospy.Subscriber(self.IMAGE_TOPIC, Image, callback=self.image_callback)
        self.pub_drive = rospy.Publisher(self.DRIVE_TOPIC, AckermannDriveStamped, queue_size=1)
        self.last_drive = AckermannDriveStamped()
    
    def image_callback(self, msg):
        self.last_image = msg.data
        
    def show_last_image(self):
        im = np.fromstring(self.last_image,dtype=np.uint8).reshape((480,-1,3))[...,::-1]
        return im
        
    def scan_callback(self, msg):
        self.last_scan = msg.ranges
        
    def drive(self, speed, angle):
        msg = AckermannDriveStamped()
        msg.drive.speed = speed
        msg.drive.steering_angle = angle
        self.last_drive = msg
    
    def stop(self):
        self.drive(0, 0) #self.last_drive.drive.steering_angle)
    
    def look(self):
        return self.last_image
    
    def scan(self):
        return self.last_scan
    
    def run(self, func, limit=10):
        r = rospy.Rate(60)
        t = rospy.get_time()
        video = cv2.VideoCapture(1)
        while rospy.get_time() - t < limit and not rospy.is_shutdown():
            func(video.read()[1])
            self.pub_drive.publish(self.last_drive)
            r.sleep()
        video.release()
        
        self.stop()
        self.pub_drive.publish(self.last_drive)
        time.sleep(0.1)
    
rospy.init_node('racecar')
rc = Racecar()
print('ROS node started successfully')

# Start Up Instructions

<p style='font-size:1.75rem;line-height:1.5'>
    Please read the startup instructions PDF before getting started!
    </p>

# Operating the Car

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to keep your car on your block when testing! Do NOT take it off without a TA's prior approval!</b>
</p>

<p style='font-size:1.75rem;line-height:1.5'>
    <code>rc.drive({speed}, {angle})</code> is a function that tells the car to drive at the speed and angle passed into the function. 
    <br> <code>rc.run({function name})</code> is a function that tells the car to run the function passed into <code>rc.run</code> for about 10 seconds. 
</p>

<p style='font-size:1.75rem;line-height:1.5'>
    For now, let us experiment with the speed and angle to get used to how the car measurements work. 
</p>

In [ ]:
# car will drive forward for ~10 seconds
def drive_forward(frame):
    rc.drive(0.2, 0) # change these numbers: rc.run(speed, angle)
    
rc.run(drive_forward)

# Part 1: Speed Measurements

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to get a sticker from a TA before taking your car off the block!</b>
</p>

<p style='font-size:1.75rem;line-height:1.5'>
    By now, we have realized that we need to approximate what <code>0.2</code> in car measurements translates into human measurements. To do so, we TAs have prepared a long ruler tape that the car can drive upon and based on that and a (possibly human) calculator, we can calculate the speed of car in <code>inches/sec</code>. 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    Car measurements and human measurements may not translate linearly, so it may be wise to test at different intervals:
    <ul style='font-size:1.75rem;line-height:1.5'>
        <li>To help both us document and double check, submit the results to the following form. You may submit it multiple times: https://forms.gle/iNzNW2aiQE8nFAqA7</li>
        <li>If the measurements are drastically different from what is expected, we will tell you to fix it 
            <br> (may also just be a low motor battery problem).</li>
    </ul>
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    Get a TA's okay when your group is ready to start the car on the ground. (TAs may ask you at what number a car will no longer speed up or turn any more.)
    </p>

In [ ]:
def drive_forward(frame):
    rc.run(0.2, 0)
    
rc.run(drive_forward)

# Part 2: Shape Driving

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to get a sticker from a TA before taking your car off the block!</b>
    <br> Now that we have basic idea of the measurements, let us make a function that will drive the car in a circle autonomously! 
</p>

<p style='font-size:1.75rem;line-height:1.5'>
    Drive your car in a circle autonomously!
    </p>

In [ ]:
# define your own function 

def circle(frame):
    pass 
    # use the drive function to have the car run in a circle 
    rc.drive(0.2, 0)

rc.run(circle)

<p style='font-size:1.75rem;line-height:1.5'>
    Drive your car in a rectangle autonomously!
    </p>

In [ ]:
def rectangle(frame):
    pass 
    # use the drive function to have the car run 

# rc.run()

<p style='font-size:1.75rem;line-height:1.5'>
    <b>Challenge:</b> Try driving your car in a figure-eight autonomously! 
</p> 

In [ ]:
def figure_eight(frame):
    pass 
    # use the drive function to have the car run 

# rc.run()

# Mini Competition 1: Cone Parking

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to get a sticker from a TA before taking your car off the block!</b>
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    Now, we are going to have a small competition where we learn to park cars in front of a cone. 
    <br>Here are the rules:
    <ol style='font-size:1.75rem;line-height:1.5'>
        <li> Winner is the car that <b style='color:green'>starts the furthest away from cone</b>, and <b style='color:green'>parks exactly half a foot away</b> from the cone. 
            <br>The highest scoring (aka most accurate parking) car wins.</li> 
        <br>
        <li> <b style='color:magenta'>The car should park autonomously.</b> 
            <br> Any attempt that involves the joystick taking over control will be disqualified. </li>
        <br>
        <li> <b style='color:blue'>Hitting a wall = car getting taken away.</b> <b>So be careful even on your first attempts!</b>
            <br> It is better to start off too slow than too fast. <b>START SLOW</b></li>
    </ol>
</p>

<p style='font-size:1.75rem;line-height:1.5'>
    Points calculation formula (best attempt):
    <br> <img src="https://latex.codecogs.com/svg.latex?\Large&space;points=\frac{start\_distance-\sqrt{speed}*error}{12}" title="\Large x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}" />
    <br>
</p>



In [ ]:
def known_cone_parking(frame, argument):
    pass 

# rc.run() 


# Mini Competition 2: Obstacle Navigation

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to get a sticker from a TA before taking your car off the block!</b>
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    Next, we will have a competition to see who can best navigate obstacles of exactly 4 feet apart. The winning car will be the car that can best weave through these obstacles and not bump into them. This time, we won't give a rubric.
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    Have fun testing! 
    </p>
<img src='https://sayingimages.com/wp-content/uploads/dont-worry-you-got-this-meme.jpg' height='300' width='300'/> 



In [1]:
def known_obs_nav(frame):
    pass 

# rc.run() 
